In [3]:
import cv2

def initialize_tracker(tracker_type, frame, bbox):
    if tracker_type == 'KCF':
        tracker = cv2.TrackerKCF_create()
    elif tracker_type == 'CSRT':
        tracker = cv2.TrackerCSRT_create()
    else:
        raise ValueError("Unsupported tracker type")
    tracker.init(frame, bbox)
    return tracker

def run_tracker(tracker_type, video_path, output_folder):
    cap = cv2.VideoCapture(video_path, cv2.CAP_AVFOUNDATION)

    
    ret, frame = cap.read()
    if not ret:
        print("Failed to read the video")
        cap.release()
        return
    
    bbox = cv2.selectROI(f"Select Object for {tracker_type}", frame, False)
    cv2.destroyWindow(f"Select Object for {tracker_type}")
    
    tracker = initialize_tracker(tracker_type, frame, bbox)
    
    frame_count = 0
    max_frames = 300
    while frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        
        ret, bbox = tracker.update(frame)
        if ret:
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
        else:
            cv2.putText(frame, "Tracking failure detected", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
        
        frame_count += 1
        cv2.imshow(f"Tracking with {tracker_type}", frame)
        cv2.imwrite(f"{output_folder}/{tracker_type}_frame_{frame_count}.png", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Paths
video_path = "./data/cars.mp4"
output_folder = "./output"

# Run KCF tracker
run_tracker('KCF', video_path, output_folder)

# Run CSRT tracker
run_tracker('CSRT', video_path, output_folder)